In [8]:
df = px.data.election()
df.head()


,district,Coderre,Bergeron,Joly,total,winner,result,district_id
0,101-Bois-de-Liesse,2481,1829,3024,7334,Joly,plurality,101
1,102-Cap-Saint-Jacques,2525,1163,2675,6363,Joly,plurality,102
2,11-Sault-au-Récollet,3348,2770,2532,8650,Coderre,plurality,11
3,111-Mile-End,1734,4782,2514,9030,Bergeron,majority,111
4,112-DeLorimier,1770,5933,3044,10747,Bergeron,majority,112


In [7]:
import json

colombia_geojson = json.load(open("/Users/kak4007/Documents/hackathons/cornell_health_2022/HealthHack2022/colombia.geo.json"))
colombia_geojson["features"][0]


{'type': 'Feature',
 'properties': {'DPTO': '05',
  'NOMBRE_DPT': 'ANTIOQUIA',
  'AREA': 63351855546.895,
  'PERIMETER': 1963728.843,
  'HECTARES': 6335185.555},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-76.30729675220047, 8.619299889431787],
    [-76.29810333270866, 8.616399764640406],
    [-76.2917022709176, 8.617500305932477],
    [-76.27619934020201, 8.60879993391571],
    [-76.26290130676628, 8.603500365680683],
    [-76.25189971939517, 8.593600272668734],
    [-76.2397994998317, 8.58959960902182],
    [-76.22709655816153, 8.583700180478338],
    [-76.2276992798516, 8.579099655292133],
    [-76.21730041464741, 8.571599959760825],
    [-76.21620178287812, 8.56400012950135],
    [-76.2144012458417, 8.551300048733623],
    [-76.21209716764655, 8.53460025816843],
    [-76.2115020744161, 8.521900177648424],
    [-76.20809936534768, 8.50100040489879],
    [-76.20520019535385, 8.487700462096564],
    [-76.20169830239917, 8.471599578043385],
    [-76.20059966999881, 8.46399974

In [28]:
import pandas as pd

dpts = ['amazonas', 'antioquia']

precips = pd.DataFrame()

for dpt in dpts:
    precip = pd.read_csv(f"colombia_{dpt}_precip.csv", skiprows=3)
    precip['NOMBRE_DPT'] = dpt.upper()
    precip = precip.rename(columns={'Unnamed: 0': 'year'})
    precips = pd.concat([precips, precip])
precips

precips.to_csv("precips_example.csv")

In [26]:
jul_precips = precips[['Jul','NOMBRE_DPT']].groupby('NOMBRE_DPT').mean()
jul_precips.reset_index(inplace=True)
jul_precips

,NOMBRE_DPT,Jul
0,AMAZONAS,245.456218
1,ANTIOQUIA,225.933109


In [40]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

df = px.data.election()
geojson = px.data.election_geojson()
candidates = df.winner.unique()

app = JupyterDash(__name__)

app.layout = html.Div([
    html.P("Candidate:"),
    dcc.RadioItems(
        id='candidate', 
        options=[{'value': x, 'label': x} 
                 for x in candidates],
        value=candidates[0],
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Graph(id="choropleth"),
])

@app.callback(
    Output("choropleth", "figure"), 
    [Input("candidate", "value")])
def display_choropleth(candidate):
    fig = px.choropleth(
        jul_precips, geojson=colombia_geojson, color="Jul",
        locations='NOMBRE_DPT', featureidkey="properties.NOMBRE_DPT",
        projection="mercator", range_color=[0, 6500])
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    return fig

app.run_server(debug=True)



Dash app running on http://127.0.0.1:8050/


In [48]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

df = px.data.election()
geojson = px.data.election_geojson()
candidates = df.winner.unique()

app = JupyterDash(__name__)

app.layout = html.Div(children=[
    html.Div(
        children=[html.H2(children="Colombia Dashboard", className='h2-title'),],
        className='study-browser-banner row'
    ),
    html.Div(
        className='row app-body',
        children=[dcc.Graph()])
])

def display_choropleth():
    fig = px.choropleth(
        jul_precips, geojson=colombia_geojson, color="Jul",
        locations='NOMBRE_DPT', featureidkey="properties.NOMBRE_DPT",
        projection="mercator", range_color=[0, 6500])
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    return fig

app.run_server(debug=True)



Dash app running on http://127.0.0.1:8050/


In [37]:
app.layout = html.Div(children=[
    html.Div(
        children=[html.H2(children="Colombia Dashboard", className='h2-title'),],
        className='study-browser-banner row'
    ),
    html.Div(
        className='row app-body',
        children=[


            dcc.Graph(
                id='map-plot3',
                figure={
                    'data': [go.Choroplethmapbox(
                        z = [1,2],
                        geojson=counties,
                        locations=locations,
                        colorscale='Viridis',
                        colorbar_title="Precipitation"
                    )],
                    'layout': go.Layout(
                        mapbox_style="carto-positron",
                        mapbox_accesstoken=token,
                        mapbox_zoom=4,
                        mapbox_center = {"lat": 4.570868, "lon": -74.2973328}
                    )
                }
            )
        ])])

app.run_server(mode='inline')

NameError: name 'go' is not defined